# TNT Cates (Living Polymers): Stress Relaxation

## Objectives
- Fit TNTCates model to stress relaxation data
- Show exponential decay in fast-breaking limit
- Extract tau_d from relaxation modulus G(t)
- Compare with TNTSingleMode exponential decay
- Perform Bayesian inference with NUTS

## Setup

In [ ]:
import os
import sys
import time

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import numpy as np
import matplotlib.pyplot as plt
import arviz as az
from scipy.optimize import curve_fit

from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64
verify_float64()

from rheojax.models.tnt import TNTCates

sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    load_laponite_relaxation,
    compute_fit_quality,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
    get_tnt_cates_param_names,
    compute_cates_tau_d,
)

param_names = get_tnt_cates_param_names()
print(f"TNTCates parameters: {param_names}")

## Theory: Cates Model for Stress Relaxation

The Cates model predicts stress relaxation following step strain:

**Relaxation modulus:**
In the fast-breaking limit ($\tau_{\text{break}} \ll \tau_{\text{rep}}$):
$$G(t) \approx G_0 \exp(-t/\tau_d) + G_\infty$$

where $\tau_d = \sqrt{\tau_{\text{rep}} \cdot \tau_{\text{break}}}$ is the effective relaxation time.

**Comparison with Maxwell:**
- TNTSingleMode: $G(t) = G_0 \exp(-t/\tau)$ (purely reptation)
- TNTCates: Same exponential form but $\tau_d$ combines reptation and breaking

**Physical interpretation:**
- Faster breaking → shorter $\tau_d$ → faster relaxation
- $\zeta = \tau_{\text{break}}/\tau_{\text{rep}} \ll 1$ for fast-breaking

## Load Stress Relaxation Data

In [ ]:
time_data, G_t = load_laponite_relaxation(aging_time=1800)

print(f"Data points: {len(time_data)}")
print(f"Time range: {time_data.min():.2e} to {time_data.max():.2e} s")
print(f"Modulus range: {G_t.min():.2e} to {G_t.max():.2e} Pa")

fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(time_data, G_t, 'o', label='Experimental data', markersize=5)
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Relaxation Modulus G(t) (Pa)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_title('Stress Relaxation Data', fontsize=14)
plt.show()
plt.close('all')

## NLSQ Fitting

In [ ]:
model = TNTCates()

start_time = time.time()
model.fit(time_data, G_t, test_mode='relaxation', method='scipy')
fit_time = time.time() - start_time

print(f"\nNLSQ Optimization completed in {fit_time:.2f} seconds")

# Extract fitted parameters
nlsq_params = {name: model.parameters.get_value(name) for name in param_names}
print("\nNLSQ Parameters:")
for name, value in nlsq_params.items():
    print(f"  {name}: {value:.4e}")

# Compute fit quality
G_pred_fit = model.predict(time_data, test_mode='relaxation')
quality = compute_fit_quality(G_t, G_pred_fit)
print(f"\nFit Quality: R² = {quality['R2']:.6f}, RMSE = {quality['RMSE']:.4e}")

## Visualize NLSQ Fit

In [ ]:
time_pred = jnp.logspace(jnp.log10(time_data.min()), jnp.log10(time_data.max()), 300)
G_pred = model.predict(time_pred, test_mode='relaxation')

# Compute residuals
residuals = G_t - model.predict(time_data, test_mode='relaxation')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.loglog(time_data, G_t, 'o', label='Data', markersize=5)
ax1.loglog(time_pred, G_pred, '-', linewidth=2, label='TNTCates fit')
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('G(t) (Pa)', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_title('Relaxation Modulus Fit', fontsize=14)

ax2.semilogx(time_data, residuals, 'o', markersize=4)
ax2.axhline(0, color='k', linestyle='--', linewidth=1)
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('Residuals (Pa)', fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.set_title('Residuals', fontsize=14)

plt.tight_layout()
plt.show()
plt.close('all')

## Physical Analysis: Extract tau_d from Exponential Fit

In [ ]:
tau_d_cates = compute_cates_tau_d(nlsq_params['tau_rep'], nlsq_params['tau_break'])
zeta = nlsq_params['tau_break'] / nlsq_params['tau_rep']

def exponential_decay(t, G0, tau, Ginf):
    return G0 * np.exp(-t / tau) + Ginf

popt, _ = curve_fit(
    exponential_decay,
    np.array(time_data),
    np.array(G_t),
    p0=[nlsq_params['G_0'], tau_d_cates, nlsq_params['eta_s']*1e-6],
    maxfev=5000
)
G0_exp, tau_d_exp, Ginf_exp = popt

print(f"\nPhysical Analysis:")
print(f"  Reptation time (tau_rep): {nlsq_params['tau_rep']:.4e} s")
print(f"  Breaking time (tau_break): {nlsq_params['tau_break']:.4e} s")
print(f"  Effective relaxation time (tau_d from Cates): {tau_d_cates:.4e} s")
print(f"  Effective relaxation time (tau_d from exp fit): {tau_d_exp:.4e} s")
print(f"  Agreement: {abs(tau_d_cates - tau_d_exp)/tau_d_cates * 100:.1f}% difference")
print(f"  Fast-breaking parameter (zeta): {zeta:.4f}")

if zeta < 0.1:
    print(f"\n  → Fast-breaking limit: Exponential decay expected")
else:
    print(f"\n  → Not in fast-breaking limit: May show deviations from exponential")

## Compare with Single-Mode Maxwell

In [ ]:
from rheojax.models.tnt import TNTSingleMode

maxwell_model = TNTSingleMode()
maxwell_model.model_params['G_0'].value = nlsq_params['G_0']
maxwell_model.model_params['tau'].value = tau_d_cates
maxwell_model.model_params['eta_s'].value = nlsq_params['eta_s']

maxwell_pred = maxwell_model.predict(time_pred, test_mode='relaxation')

fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(time_data, G_t, 'o', label='Data', markersize=5, zorder=3)
ax.loglog(time_pred, G_pred, '-', linewidth=2, label='TNTCates', zorder=2)
ax.loglog(time_pred, maxwell_pred, '--', linewidth=2, label=f'Maxwell (τ={tau_d_cates:.2e}s)', zorder=1)
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('G(t) (Pa)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_title('Cates vs Maxwell Comparison', fontsize=14)
plt.show()
plt.close('all')

## Bayesian Inference with NUTS

In [ ]:
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

print(f"Starting Bayesian inference with NUTS...")
print(f"Warmup: {NUM_WARMUP}, Samples: {NUM_SAMPLES}, Chains: {NUM_CHAINS}")

start_time = time.time()
bayesian_result = model.fit_bayesian(
    time_data, G_t,
    test_mode='relaxation',
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    seed=42
)
bayes_time = time.time() - start_time

print(f"\nBayesian inference completed in {bayes_time:.2f} seconds")
print(f"Time per sample: {bayes_time/(NUM_WARMUP + NUM_SAMPLES):.3f} seconds")

## Convergence Diagnostics

In [ ]:
posterior = bayesian_result.posterior_samples

bayesian_params = {name: float(jnp.mean(posterior[name])) for name in param_names}
param_std = {name: float(jnp.std(posterior[name])) for name in param_names}

print("\nPosterior Statistics:")
for name in param_names:
    print(f"  {name}: {bayesian_params[name]:.4e} ± {param_std[name]:.4e}")

# Compare NLSQ vs Bayesian using the utility function
print_parameter_comparison(model, posterior, param_names)

## ArviZ Trace Plot

In [ ]:
idata = az.from_dict(posterior={k: v.reshape(NUM_CHAINS, NUM_SAMPLES) for k, v in posterior.items()})

axes = az.plot_trace(idata, var_names=param_names, compact=False, figsize=(12, 8))
plt.suptitle('MCMC Trace Plots', fontsize=14, y=1.0)
plt.tight_layout()
plt.show()
plt.close('all')

## ArviZ Pair Plot

In [ ]:
axes = az.plot_pair(
    idata,
    var_names=param_names,
    kind='kde',
    marginals=True,
    figsize=(10, 10)
)
plt.suptitle('Posterior Correlations', fontsize=14, y=0.995)
plt.show()
plt.close('all')

## ArviZ Forest Plot

In [ ]:
axes = az.plot_forest(
    idata,
    var_names=param_names,
    combined=True,
    hdi_prob=0.95,
    figsize=(8, 4)
)
plt.suptitle('95% Credible Intervals', fontsize=14)
plt.show()
plt.close('all')

## Posterior Predictive Distribution

In [ ]:
n_posterior_samples = 200
sample_indices = np.random.choice(NUM_SAMPLES * NUM_CHAINS, n_posterior_samples, replace=False)

predictions = []
for idx in sample_indices:
    # Set model parameters from posterior sample
    for j, name in enumerate(param_names):
        model.parameters.set_value(name, float(posterior[name].flatten()[idx]))
    # Predict with updated parameters
    pred_i = model.predict(time_pred, test_mode='relaxation')
    predictions.append(np.array(pred_i))

predictions = np.array(predictions)
pred_mean = np.mean(predictions, axis=0)
pred_lower = np.percentile(predictions, 2.5, axis=0)
pred_upper = np.percentile(predictions, 97.5, axis=0)

fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(time_data, G_t, 'o', label='Data', markersize=5, zorder=3)
ax.loglog(time_pred, pred_mean, '-', linewidth=2, label='Posterior mean', zorder=2)
ax.fill_between(time_pred, pred_lower, pred_upper, alpha=0.3, label='95% CI', zorder=1)
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('G(t) (Pa)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_title('Posterior Predictive Distribution', fontsize=14)
plt.show()
plt.close('all')

## Physical Interpretation from Posterior

In [ ]:
tau_d_posterior = np.sqrt(posterior['tau_rep'] * posterior['tau_break'])
zeta_posterior = posterior['tau_break'] / posterior['tau_rep']

print(f"\nPhysical quantities from posterior:")
print(f"  tau_d: {np.mean(tau_d_posterior):.4e} ± {np.std(tau_d_posterior):.4e} s")
print(f"  zeta (tau_break/tau_rep): {np.mean(zeta_posterior):.4f} ± {np.std(zeta_posterior):.4f}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.hist(tau_d_posterior, bins=30, alpha=0.7, edgecolor='black')
ax1.axvline(np.mean(tau_d_posterior), color='r', linestyle='--', label='Mean')
ax1.set_xlabel(r'$\tau_d$ (s)', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.legend()
ax1.set_title('Effective Relaxation Time Distribution', fontsize=12)

ax2.hist(zeta_posterior, bins=30, alpha=0.7, edgecolor='black')
ax2.axvline(np.mean(zeta_posterior), color='r', linestyle='--', label='Mean')
ax2.axvline(0.1, color='g', linestyle=':', label='Fast-breaking limit')
ax2.set_xlabel(r'$\zeta = \tau_{break}/\tau_{rep}$', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.legend()
ax2.set_title('Fast-Breaking Parameter Distribution', fontsize=12)

plt.tight_layout()
plt.show()
plt.close('all')

## Save Results

In [ ]:
save_tnt_results(model, bayesian_result, "cates", "relaxation", param_names)

## Key Takeaways

1. **Exponential relaxation** in Cates model arises from effective time scale $\tau_d = \sqrt{\tau_{\text{rep}} \cdot \tau_{\text{break}}}$
2. **Fast-breaking limit** gives Maxwell-like exponential decay: $G(t) \sim \exp(-t/\tau_d)$
3. **Direct extraction** of $\tau_d$ from exponential fit agrees with Cates parameters
4. **Physical equivalence** with single-mode Maxwell but with different microscopic origin (breaking vs pure reptation)
5. **Bayesian inference** provides uncertainty quantification for time scales from relaxation data

**Next steps:** Compare $\tau_d$ across protocols (flow curve, startup, SAOS) to verify consistency.